In [ ]:
# 1. Carregar o Dataset e Exploração Inicial

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import joblib

# Caminho para o ficheiro (ajusta conforme o teu caso)
ficheiro = r"C:\Users\Guilherme\Desktop\Redes de dados\Projeto\Dataset\Dataset2\dataset2_ircBot_hajime.xlsx"

# Carregar o datasetS
df = pd.read_excel(ficheiro)

# Mostra as primeiras 
print("Primeiras linhas do dataset:")
print(df.head())

# Verificar as colunas disponíveis
print("\nColunas disponíveis:", list(df.columns))

# Análise de classes (para confirmar se está equilibrado)
print("\nDistribuição da coluna 'Label':")
print(df['Label'].value_counts())

# Verificar tipos de dados e valores nulos
print("\nValores em falta por coluna:")
print(df.isnull().sum())

# 2. Pré-processamento dos Dados

# (Opcional) Codificar 'Protocol' se existir
if 'Protocol' in df.columns:
    df['Protocol_num'] = LabelEncoder().fit_transform(df['Protocol'])
    features = ['Length', 'Protocol_num']
else:
    features = ['Length']

# Definir X e y
X = df[features].values
y = df['Label'].values

# Normalizar as features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Guardar scaler
joblib.dump(scaler, "scaler_ann_iot23.gz")

# Dividir treino/validação (80%/20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
# 3. Criar e Treinar a ANN

def criar_modelo(input_dim, n_neuronios=16):
    model = tf.keras.Sequential([
        layers.Dense(n_neuronios, activation='relu', input_shape=(input_dim,)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Parâmetros do modelo
NUM_TREINOS = 3
NUM_EPOCAS = 15
BATCH_SIZE = 16

# Guardar históricos e métricas
historicos = []
scores = []
for i in range(NUM_TREINOS):
    print(f"\nTreino {i+1} de {NUM_TREINOS}")
    modelo = criar_modelo(X_train.shape[1])
    historico = modelo.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=NUM_EPOCAS,
        batch_size=BATCH_SIZE,
        verbose=1
    )
    historicos.append(historico)
    # Previsão em validação
    y_pred = (modelo.predict(X_val) > 0.5).astype(int).flatten()
    acc = accuracy_score(y_val, y_pred)
    scores.append(acc)
    print(f"Accuracy do treino {i+1}: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_val, y_pred))

# 4. Visualização dos Resultados

plt.figure(figsize=(10, 6))
for i, h in enumerate(historicos):
    plt.plot(h.history['val_accuracy'], label=f'Treino {i+1}', alpha=0.6)
plt.title('Accuracy de Validação por Época')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

# Accuracy média e desvio padrão
print("\nAccuracies em cada treino:", scores)
print("Accuracy média:", np.mean(scores))
print("Accuracy desvio padrão:", np.std(scores))

# Avaliação detalhada do último treino
y_pred_final = (modelo.predict(X_val) > 0.5).astype(int).flatten()
print("\nMatriz de Confusão (último treino):\n", confusion_matrix(y_val, y_pred_final))

# 5. Guardar Modelo Treinado

modelo.save("modelo_ann_iot23.h5")
print("\nModelo guardado em 'modelo_ann_iot23.h5'")




Primeiras linhas do dataset:
           Time           Source     Destination Protocol  Length  \
0   7675.538342   35.157.240.102   192.168.1.158  TLSv1.2     875   
1  20371.259680  192.168.100.111   147.32.57.107      TCP      60   
2  26145.265144  192.168.100.111   112.28.138.95      TCP      60   
3  54364.382321    192.168.1.158  35.157.240.102      TCP      60   
4  56559.046524    192.168.1.158  35.157.240.102      TCP      60   

                                                Info  Label  
0                                   Application Data      0  
1              24  >  23 [SYN] Seq=0 Win=14600 Len=0      1  
2           27824  >  81 [SYN] Seq=0 Win=14600 Len=0      1  
3  [TCP Dup ACK 76575#1] 28291  >  443 [ACK] Seq=...      0  
4  30367  >  443 [ACK] Seq=2189969 Ack=5662275 Wi...      0  

Colunas disponíveis: ['Time', 'Source', 'Destination', 'Protocol', 'Length', 'Info', 'Label']

Distribuição da coluna 'Label':
Label
0    303605
1    303605
Name: count, dtype: int64


c:\Users\Guilherme\Desktop\Redes de dados\Projeto\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30361/30361 ━━━━━━━━━━━━━━━━━━━━ 27s 885us/step - accuracy: 0.6614 - loss: 0.5999 - val_accuracy: 0.6677 - val_loss: 0.5886
Epoch 2/15
30084/30361 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.6696 - loss: 0.5871

In [2]:
# =============================================
# 7. Simulação IDS — Teste com nomes reais de protocolo
# =============================================

from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np
import pandas as pd

# Carregar o modelo e scaler treinados
modelo = load_model("modelo_ann_iot23.h5")
scaler = joblib.load("scaler_ann_iot23.gz")

ficheiro = r"C:\Users\Guilherme\Desktop\Redes de dados\Projeto\Dataset\Dataset2\dataset2_ircBot_hajime.xlsx"
df = pd.read_excel(ficheiro)

# Criar o LabelEncoder
encoder = LabelEncoder()
encoder.fit(df['Protocol'])  

# Mostrar os códigos dos protocolos
protocol_map = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("Códigos dos protocolos:", protocol_map)

# Exemplos reais de previsão
exemplos = [
    {'length': 60, 'protocol': 'UDP'},
    {'length': 1200, 'protocol': 'TCP'}, 
    {'length': 250, 'protocol': 'HTTP'}
]

# Prever vários exemplos
for i, ex in enumerate(exemplos):
    length_exemplo = ex['length']
    protocolo_exemplo = ex['protocol']
    if protocolo_exemplo in protocol_map:
        protocol_num = protocol_map[protocolo_exemplo]
        novos_dados = np.array([[length_exemplo, protocol_num]])
        novos_dados_norm = scaler.transform(novos_dados)
        y_pred = (modelo.predict(novos_dados_norm) > 0.5).astype(int)
        print(f"\nExemplo {i+1}: Length={length_exemplo}, Protocol={protocolo_exemplo} ({protocol_num}) => Predição:", y_pred.flatten())
        if y_pred[0][0] == 1:
            print("  → O modelo detetou: **ATAQUE** (1)")
        else:
            print("  → O modelo detetou: **BENIGNO** (0)")
    else:
        print(f"\nExemplo {i+1}: Protocol '{protocolo_exemplo}' não encontrado no encoding! Tenta outro protocolo.")



Códigos dos protocolos: {'ARP': np.int64(0), 'BT-DHT': np.int64(1), 'CDP': np.int64(2), 'DHCP': np.int64(3), 'DNS': np.int64(4), 'HTTP': np.int64(5), 'ICMP': np.int64(6), 'ICMPv6': np.int64(7), 'IGMPv2': np.int64(8), 'LOOP': np.int64(9), 'NTP': np.int64(10), 'SSDP': np.int64(11), 'SSH': np.int64(12), 'SSHv2': np.int64(13), 'TCP': np.int64(14), 'TELNET': np.int64(15), 'TLSv1.2': np.int64(16), 'UAUDP': np.int64(17), 'UDP': np.int64(18)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

Exemplo 1: Length=60, Protocol=UDP (18) => Predição: [0]
  → O modelo detetou: **BENIGNO** (0)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Exemplo 2: Length=1200, Protocol=TCP (14) => Predição: [1]
  → O modelo detetou: **ATAQUE** (1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Exemplo 3: Length=250, Protocol=HTTP (5) => Predição: [1]
  → O modelo detetou: **ATAQUE** (1)
